In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from Dataset_Loader import getData
from Neural_Network_Model import SingleNeuronModel, FullyConnectedModel
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, matthews_corrcoef

In [27]:
import warnings
warnings.filterwarnings('ignore') 

In [7]:
def train_model(model, optimizer, criterion, X, y, logdir, epochs=100):
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0  # Initialize total loss for the epoch
        i = 0
        total_accuracy = 0
        writer = SummaryWriter(log_dir = logdir)
        for batch in train_loader:
            # Extract inputs and labels from the batch
            X_batch, y_batch = batch['x'], batch['y']
            
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(X_batch)
            
            # Compute loss
            loss = criterion(output, y_batch)

            # L2 regularization for neuron weights
            l2_reg = torch.tensor(0.)
            for param in model.parameters():
                if len(param.size()) > 1:  # Check if the parameter is a weight (not bias)
                    l2_reg += torch.norm(param, p=2)
            loss += 0.01 * l2_reg  # Adjust the regularization strength as needed

            # Backward pass
            loss.backward()

            # Update weights
            optimizer.step()
            
            total_loss += loss.item()
            
            with torch.no_grad():
                predictions = (model(X_batch).numpy() > 0.5).astype(int)
            
            accuracy = accuracy_score(y_batch, predictions)
            total_accuracy += accuracy
            
            roc_auc = roc_auc_score(y_batch, predictions)
            pr_auc = average_precision_score(y_batch, predictions)
            mcc = matthews_corrcoef(y_batch, predictions)

            i += 1
        
            
        for name, param in model.named_parameters():
           writer.add_histogram(f'{name}', param, epoch)
        

        average_loss = total_loss / i
        
        writer.add_scalar("Train Loss", average_loss, epoch)
        writer.add_scalar("Train Accuracy", total_accuracy/i, epoch)
        writer.add_scalar('Metrics/ROC_AUC', roc_auc, epoch)
        writer.add_scalar('Metrics/PR_AUC', pr_auc, epoch)
        writer.add_scalar('Metrics/MCC', mcc, epoch)

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {average_loss:.4f}') 
        writer.flush()
        
    writer.close()
        

    return model

In [42]:
def evaluate_model(model, test_data_loader):
    i = 0
    total_accuracy = 0
    for batch in test_data_loader:
        X_batch, y_batch = batch['x'], batch['y']
        with torch.no_grad():
            predictions = (model(X_batch).numpy() > 0.5).astype(int)
        i += 1
        roc_auc = roc_auc_score(y_batch, predictions)
        pr_auc = average_precision_score(y_batch, predictions)
        mcc = matthews_corrcoef(y_batch, predictions)
    
    return {"roc_auc": roc_auc / i, "pr_auc": pr_auc, "mcc": mcc}

In [4]:
hidden_sizes = [32, 64, 128]

### Load Data

In [8]:
batch_size = 1024

In [9]:
use_weighted_sampler = True  # Set to True to use WeightedRandomSampler
train_loader, test_loader = getData(batch_size, use_weighted_sampler)

### Single Neuron Model

In [10]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
device

device(type='cuda')

In [22]:
torch.cuda.is_available()

True

In [23]:
input_size = 2

In [24]:
single_neuron_model = SingleNeuronModel(input_size)
optimizer = optim.SGD(single_neuron_model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [25]:
for name, param in single_neuron_model.named_parameters():
    print(name)

linear.weight
linear.bias


In [28]:
single_neuron_model = train_model(single_neuron_model, optimizer, criterion, train_loader, test_loader,'models/run1', epochs = 10)

Epoch [1/10], Loss: 50.0150
Epoch [2/10], Loss: 50.0393
Epoch [3/10], Loss: 50.2733
Epoch [4/10], Loss: 50.3617
Epoch [5/10], Loss: 50.3424
Epoch [6/10], Loss: 50.3672
Epoch [7/10], Loss: 50.3320
Epoch [8/10], Loss: 50.3118
Epoch [9/10], Loss: 50.3781
Epoch [10/10], Loss: 50.2763


In [43]:
results = evaluate_model(single_neuron_model, test_loader)
print(results)

{'roc_auc': 0.0017667844522968198, 'pr_auc': 0.038834951456310676, 'mcc': 0.0}


### Fully Connected Model with 32 hidden layers and optimizer as ADAM and Loss as Binary Cross Entropy

In [30]:
hidden_size = 32
fully_connected_model_32 = FullyConnectedModel(input_size, hidden_size)
optimizer = optim.Adam(fully_connected_model_32.parameters(), lr=0.001) 
criterion = nn.BCELoss() 

In [31]:
fully_connected_model_32 = train_model(fully_connected_model_32, optimizer, criterion, train_loader, test_loader, logdir='models/run2', epochs = 10)

Epoch [1/10], Loss: 50.0591
Epoch [2/10], Loss: 50.0023
Epoch [3/10], Loss: 32.0992
Epoch [4/10], Loss: 26.2781
Epoch [5/10], Loss: 17.6702
Epoch [6/10], Loss: 0.7406
Epoch [7/10], Loss: 30.0350
Epoch [8/10], Loss: 47.2259
Epoch [9/10], Loss: 1.5239
Epoch [10/10], Loss: 41.1937
Fully Connected Model with 32 Neurons Accuracy: 0.0470


In [44]:
results = evaluate_model(fully_connected_model_32, test_loader)
print(results)

{'roc_auc': 0.0017667844522968198, 'pr_auc': 0.043689320388349516, 'mcc': 0.0}


### Fully Connected Model with 64 hidden layers and optimizer as SGD and Loss as BCEWithLogitsLoss


In [32]:
hidden_size = 64
fully_connected_model_64 = FullyConnectedModel(input_size, hidden_size)
optimizer = optim.Adam(fully_connected_model_64.parameters(), lr=0.001) 
criterion = nn.BCEWithLogitsLoss() 

In [33]:
fully_connected_model_64 = train_model(fully_connected_model_64, optimizer, criterion, train_loader, test_loader, logdir='models/run3', epochs = 10)

Epoch [1/10], Loss: 0.8200
Epoch [2/10], Loss: 0.8134
Epoch [3/10], Loss: 0.8135
Epoch [4/10], Loss: 0.8071
Epoch [5/10], Loss: 0.8144
Epoch [6/10], Loss: 0.8106
Epoch [7/10], Loss: 0.8142
Epoch [8/10], Loss: 0.8153
Epoch [9/10], Loss: 0.8140
Epoch [10/10], Loss: 0.8141
Fully Connected Model with 64 Neurons Accuracy: 0.0471


In [45]:
results = evaluate_model(fully_connected_model_64, test_loader)
print(results)

{'roc_auc': 0.0017667844522968198, 'pr_auc': 0.03640776699029126, 'mcc': 0.0}


### Fully Connected Model with 128 hidden layers and optimizer as Adagrad and Loss as BCEWithLogitsLoss


In [34]:
hidden_size = 128
fully_connected_model_128 = FullyConnectedModel(input_size, hidden_size)
optimizer = optim.Adagrad(fully_connected_model_128.parameters(), lr=0.001) 
criterion = nn.BCEWithLogitsLoss() 

In [35]:
fully_connected_model_128 = train_model(fully_connected_model_128, optimizer, criterion, train_loader, test_loader,logdir='models/run3', epochs = 10)

Epoch [1/10], Loss: 0.8743
Epoch [2/10], Loss: 0.8658
Epoch [3/10], Loss: 0.8599
Epoch [4/10], Loss: 0.8551
Epoch [5/10], Loss: 0.8509
Epoch [6/10], Loss: 0.8471
Epoch [7/10], Loss: 0.8444
Epoch [8/10], Loss: 0.8422
Epoch [9/10], Loss: 0.8396
Epoch [10/10], Loss: 0.8374
Fully Connected Model with 128 Neurons Accuracy: 0.0470


In [46]:
results = evaluate_model(fully_connected_model_128, test_loader)
print(results)

{'roc_auc': 0.0017667844522968198, 'pr_auc': 0.055825242718446605, 'mcc': 0.0}
